# Bitcoin Notebook Agent Demo

This notebook demonstrates a robust utility layer enabling an agent to operate over `.ipynb` files: parsing, dependency analysis, selective execution, caching, error repair suggestions, and advisory generation integration.

Sections follow the required outline.

## 1. Environment Setup and Dependency Installation
Install required packages (run once). Safe to skip if environment already prepared.

In [ ]:
# (Optional) Install runtime dependencies. Comment out after first run.
import sys, subprocess, textwrap
pkgs = [
    'nbformat', 'nbclient', 'rich', 'tqdm', 'black', 'openai', 'google-generativeai'
]
for p in pkgs:
    try:
        __import__(p.split('==')[0])
    except ImportError:
        print(f'Installing {p}...')
        subprocess.run([sys.executable, '-m', 'pip', 'install', p], check=False)
print('Environment check complete.')

## 2. Load and Parse .ipynb File Structure
We load a target notebook using nbformat and inspect cell metadata.

In [ ]:
import nbformat, os, json, hashlib, ast, time
from pathlib import Path
from typing import List, Dict, Any, Tuple
from rich import print as rprint

TARGET_NOTEBOOK = os.environ.get('TARGET_NOTEBOOK', 'agents/pipeline_agent.py')  # can also point to a .ipynb

def load_notebook(path: str):
    if path.endswith('.ipynb'):
        with open(path, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)
        rprint(f"[bold green]Loaded notebook with {len(nb.cells)} cells[/bold green]")
        return nb
    else:
        rprint('[yellow]Target is a .py file; wrapping into pseudo-notebook cells.[/yellow]')
        with open(path, 'r', encoding='utf-8') as f:
            code = f.read()
        return nbformat.v4.new_notebook(cells=[nbformat.v4.new_code_cell(code)])

nb_obj = load_notebook(TARGET_NOTEBOOK)
nbformat_minor = nb_obj.get('nbformat_minor', 5)
rprint({'nbformat': nb_obj.get('nbformat', 4), 'nbformat_minor': nbformat_minor})

## 3. Extract Code Cells and Metadata
Filter only code cells and capture execution counts, ids, tags.

In [ ]:
def extract_code_cells(nb) -> List[Dict[str, Any]]:
    code_cells = []
    for idx, cell in enumerate(nb.cells):
        if cell.get('cell_type') == 'code':
            code_cells.append({
                'index': idx,
                'id': cell.get('id'),
                'execution_count': cell.get('execution_count'),
                'source': cell.get('source', ''),
                'tags': cell.get('metadata', {}).get('tags', [])
            })
    return code_cells

code_cells = extract_code_cells(nb_obj)
print(f"Discovered {len(code_cells)} code cells")
code_cells[:2]

## 4. Normalize and Clean Cell Source Code
We standardize whitespace and optionally format with `black` (if installed).

In [ ]:
def normalize_source(src: str, do_format: bool = True) -> str:
    cleaned = '\n'.join(line.rstrip() for line in src.replace('\r\n', '\n').split('\n'))
    if do_format:
        try:
            import black
            cleaned = black.format_str(cleaned, mode=black.Mode())
        except Exception:
            pass
    return cleaned

for c in code_cells:
    c['normalized_source'] = normalize_source(c['source'])

code_cells[0]['normalized_source'][:300] if code_cells else 'No code cells'

## 5. Build Cell Dependency Graph (Imports and Variables)
Parse AST to gather imported modules and top-level assigned symbols per cell.

In [ ]:
def analyze_cell_symbols(src: str) -> Tuple[set, set]:
    imports, assigns = set(), set()
    try:
        tree = ast.parse(src)
    except Exception:
        return imports, assigns
    for node in ast.walk(tree):
        if isinstance(node, (ast.Import, ast.ImportFrom)):
            for n in node.names:
                imports.add(n.name.split('.')[0])
        elif isinstance(node, ast.Assign):
            for t in node.targets:
                if isinstance(t, ast.Name):
                    assigns.add(t.id)
                elif isinstance(t, (ast.Tuple, ast.List)):
                    for elt in t.elts:
                        if isinstance(elt, ast.Name):
                            assigns.add(elt.id)
        elif isinstance(node, ast.FunctionDef):
            assigns.add(node.name)
        elif isinstance(node, ast.ClassDef):
            assigns.add(node.name)
    return imports, assigns

for c in code_cells:
    imps, defs = analyze_cell_symbols(c['normalized_source'])
    c['imports'] = sorted(imps)
    c['defines'] = sorted(defs)

dep_index = {c['index']: {'imports': c['imports'], 'defines': c['defines']} for c in code_cells}
list(dep_index.items())[:3]

## 6. Assemble Notebook Context for Agent Prompt
We build a trimmed context window summarizing each cell for LLM prompting.

In [ ]:
def cell_hash(src: str) -> str:
    return hashlib.sha256(src.encode('utf-8')).hexdigest()[:12]

MAX_PREVIEW_LINES = 12

def assemble_context(cells: List[Dict[str, Any]]) -> str:
    blocks = []
    for c in cells:
        lines = c['normalized_source'].split('\n')
        preview = '\n'.join(lines[:MAX_PREVIEW_LINES])
        blocks.append(
            f"CELL {c['index']} | id={c.get('id')} | hash={cell_hash(c['normalized_source'])}\n"
            f"Defines: {', '.join(c['defines']) or '-'} | Imports: {', '.join(c['imports']) or '-'}\n"
            f"---\n{preview}\n"
        )
    return '\n'.join(blocks)

context_snapshot = assemble_context(code_cells)
print(context_snapshot[:1000])